### 1. 환경 설정

In [3]:
#uv init -p 3.12.10
#uv add python-dotenv
#uv add langchain
#uv add langchain-ollama
#uv add langchain-qdrant
#uv add langchain-community
#uv add pypdf

In [ ]:
from dotenv import load_dotenv
load_dotenv(override=True)


In [18]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

### 2. Document Loader
- PyPDF로 문서에서 텍스트 추출

In [ ]:
loader = PyPDFLoader(file_path="data/arxiv_paper.pdf")
docs = loader.load()
len(docs)

In [ ]:
for doc in docs:
    print(doc.page_content[:500])
    print(doc.metadata)
    print("-"*100)

### 3. Text Splitter
-  RecursiveCharacterTextSplitter 사용

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

print(splits[0].page_content)

In [ ]:
[len(chunk.page_content) for chunk in splits]

### 4. Embedding and Vector Store
- 텍스트를 벡터로 변환
- Qdrant Vector Store에 저장

In [ ]:
embeddings = OllamaEmbeddings(model="bge-m3") # 벡터 모델 선택 (bge-m3)
embeddings

In [ ]:
from langchain_qdrant import QdrantVectorStore, RetrievalMode
from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams, Distance
from uuid import uuid4

# Qdrant 클라이언트 생성 (메모리형)
client = QdrantClient(":memory:")

# 컬렉션 생성
client.create_collection(
    collection_name="rag_collection",
    vectors_config=VectorParams(size=1024, distance=Distance.COSINE), # 벡터 크기(임베딩 모델에 따라 틀림)와 거리 측정 방식(코사인 거리)
)

# Vector Store 생성
vectorstore = QdrantVectorStore(
    client=client,
    collection_name="rag_collection",
    embedding=embeddings,
    retrieval_mode=RetrievalMode.DENSE, # 데이터 검색 방식 (DENSE-기본, SPARSE, HYBRID) 
)

uuids = [str(uuid4()) for _ in range(len(splits))]

# 데이터 저장
vectorstore.add_documents(
    documents=splits,
    ids=uuids,
)

### 5. Retrieval
- 데이터 검색

In [ ]:
retriever = vectorstore.as_retriever()

search_result = retriever.invoke("Embodied Agent가 뭐야?", k=5)

for doc in search_result:
    print(doc.page_content[:500])
    print(doc.metadata)
    print("-"*100)

In [14]:
# RAG Prompt Template 설정
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
당신은 Q&A 전문 AI 어시스턴트입니다. 주어진 컨텍스트를 사용하여 질문에 답변해주세요.

컨텍스트:
{context}

질문:
{question}

답변:
 """)

In [ ]:
# 단순 답변 Streaming

from IPython.display import Markdown
from langchain_core.runnables import RunnableParallel

llm = ChatOllama(model="gemma3:4b", temperature=0)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

for chunk in rag_chain.stream("Embodied Agent가 뭐야?"):
    print(chunk, end="", flush=True)

In [ ]:
# 참고한 소스를 함께 스트리밍

rag_chain_from_docs = (
    # 입력 데이터에서 context 필드를 추출하여 format_docs 함수로 처리
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    # 포맷된 컨텍스트와 질문을 프롬프트에 전달
    | prompt
    # 프롬프트의 출력을 LLM에 전달
    | llm
    # 출력을 문자열로 파싱
    | StrOutputParser()
)

# 검색 결과와 질문을 병렬로 처리하고 답변을 생성하는 최종 체인
rag_chain_with_source = RunnableParallel(
    # 검색기(retriever)로 문서를 가져오고, 질문을 그대로 전달
    {"context": retriever, "question": RunnablePassthrough()},
# 위에서 생성된 context와 question을 rag_chain_from_docs에 전달하여 answer 필드 추가   
).assign(answer=rag_chain_from_docs)

for chunk in rag_chain_with_source.stream("Embodied Agent가 뭐야?"):
    if "context" in chunk:
        print("Retrieved Documents:")
        for i, doc in enumerate(chunk["context"]):
            print(f"소스 [{i+1}]: <{doc.metadata['title']}>")
            print(f"참고내용: {doc.page_content[:100]}...\n")
        print("-"*80)
    elif "answer" in chunk:
        print(chunk["answer"], end="", flush=True)